==== Anlage des Spark Context ====

Spark-Session erstellen und notwendige Frameworks importieren

In [1]:
import findspark
findspark.init()
import pyspark
 

conf = pyspark.SparkConf()
#conf.set("spark.driver.bindAddress", "127.0.0.1")  # Localhost
#conf = conf.setMaster("local[*]")
conf = conf.setAppName("App")


sc = pyspark.SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/03 11:56:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


==== Vorverarbeitung der Datenquelle ===

In [ ]:
import pandas as pd
from datetime import datetime

# CSV-Datei laden, wobei das Pipe-Zeichen als Trennzeichen angegeben wird
csv_file_path = "data/output.csv"  # Ersetze dies mit dem Pfad zu deiner CSV-Datei
df = pd.read_csv(csv_file_path, sep='|')  # Verwende '|' als Trennzeichen

# Angenommene Spaltennummern: time = 0, icao24 = 1, lat = 2, lon = 3, onground = 8
column_indices = [0, 1, 2, 3, 8]  # Wir wählen nur die benötigten Spalten aus

# Extrahieren der angegebenen Spalten, dabei behalten wir den Header (Spaltennamen)
df_selected = df.iloc[:, column_indices]

# Umbenennen der Spalten für bessere Lesbarkeit
df_selected.columns = ['time', 'icao24', 'latitude', 'longitude', 'onground']

# Funktion zur Konvertierung des Unix-Timestamps in das gewünschte Format
def convert_timestamp(unix_timestamp):
    try:
        return datetime.utcfromtimestamp(int(unix_timestamp)).strftime('%Y/%m/%d %H:%M:%S')
    except ValueError as e:
        print(f"Fehler beim Konvertieren des Timestamps: {unix_timestamp}, Fehler: {e}")
        return None

# Umwandlung der 'time'-Spalte in das gewünschte Datumsformat
df_selected['time'] = df_selected['time'].apply(convert_timestamp)

# Umwandlung der 'onground'-Spalte, 'true' -> 1 und 'false' -> 0
df_selected['onground'] = df_selected['onground'].apply(lambda x: 1 if str(x).strip().lower() == 'true' else 0)

# Zeige die ersten 5 Zeilen des bearbeiteten DataFrames
print(df_selected.head())

# Speichere das Ergebnis als neue CSV-Datei, dabei bleibt der Header erhalten
df_selected.to_csv("data/processed_data.csv", index=False)

==== Durchführen der Datenanlyse ====

Auswertung 1 - Weltweite Flughäfen

In [2]:
import csv
from io import StringIO

def analyze_airports():
    """
    Analysiert Flughäfen weltweit und gibt die Anzahl als String zurück.
    
    :return: Formatierten String mit den Ergebnissen und das zugehörige RDD.
    """

    # Daten einlesen und Header extrahieren
    airportRDD = sc.textFile("data/airports.csv")
    header = airportRDD.first()

    # CSV korrekt parsen, um Fehler durch Kommata innerhalb von Anführungszeichen zu vermeiden
    def parse_csv(line):
        reader = csv.reader(StringIO(line))
        return next(reader)

    # Filterkriterien anwenden und nur die relevanten Spalten speichern (1, 2, 3, 4, 5, 7)
    airportRDD = (
        airportRDD.filter(lambda line: line != header)  # Header entfernen
                  .map(parse_csv)  # CSV korrekt parsen
                  .filter(lambda cols: len(cols) > 7)  # Sicherstellen, dass genug Spalten vorhanden sind
                  .map(lambda cols: (cols[1].strip('"'),  # Name
                                    cols[2].strip('"'),  # Typ
                                    cols[3].strip('"'),  # Breitengrad
                                    cols[4].strip('"'),  # Längengrad
                                    cols[5].strip('"'),  # Höhe
                                    #cols[7].strip('"')
                                    ))  # Kontinent
                  .filter(lambda cols: cols[1] in ["small_airport", "medium_airport", "large_airport", "heliport"])  # Nur bestimmte Typen
    )

    # Anzahl der Flughäfen insgesamt
    total_airports = airportRDD.count()

    # Anzahl der Flughäfen nach Kategorien
    categorized_airports = (
        airportRDD.map(lambda cols: cols[1])  # Typ (Spalte 2)
                  .countByValue()
    )

    # Formatierte Ausgabe als String erstellen
    result = "Analyse weltweiter Flughäfen und Heliports\n"
    result += "=" * 50 + "\n"

    result += f"Die Anzahl der weltweiten Flughäfen und Heliports beträgt: {total_airports}\n"
    result += f"Davon Heliports: {categorized_airports.get('heliport', 0)}\n"
    result += f"Davon kleine Flughäfen: {categorized_airports.get('small_airport', 0)}\n"
    result += f"Davon mittelgroße Flughäfen: {categorized_airports.get('medium_airport', 0)}\n"
    result += f"Davon große Flughäfen: {categorized_airports.get('large_airport', 0)}\n"

    return result, airportRDD

# Beispielaufruf und Ausgabe
result, airportRDD = analyze_airports()
print(result)

# Optional: Die Daten aus airportRDD anzeigen lassen
#airportRDD.collect()


Analyse weltweiter Flughäfen und Heliports
Die Anzahl der weltweiten Flughäfen und Heliports beträgt: 67961
Davon Heliports: 21081
Davon kleine Flughäfen: 41706
Davon mittelgroße Flughäfen: 4700
Davon große Flughäfen: 474



Auswertung 2 - Registrierte Flugobjekte

In [3]:
def analyze_aircraft_data():
    
    """
    Analysiert eine CSV-Datei mit Flugzeugdaten, um die Gesamtanzahl der registrierten Flugobjekte 
    sowie die Top 20 Airlines nach der Anzahl der registrierten Flugzeuge zu ermitteln.

    Die Funktion liest die Daten aus einer festgelegten CSV-Datei, filtert ungültige Einträge 
    und verarbeitet die Informationen mithilfe von Spark. 

    :return: Ein Tupel bestehend aus:
        - result (str): Ein formatierter String mit der Gesamtanzahl der Flugobjekte 
                        und den Top 20 Airlines nach Anzahl registrierter Flugzeuge.
        - aircraft_raw_rdd (RDD): Ein Spark RDD mit den gefilterten Flugzeugdaten.
        - aircraftRDD (RDD): Ein Spark RDD, das die Airlines und die Anzahl ihrer Flugzeuge enthält.
    """

    # Daten einlesen und Header extrahieren
    aircraft_input_rdd = sc.textFile('data/aircraft-database-complete-2024-10.csv')
    header = aircraft_input_rdd.first()

    # Filterkriterien anwenden: Entfernen des Headers
    aircraft_raw_rdd = aircraft_input_rdd.filter(lambda line: line != header)

    # Anzahl der registrierten Flugobjekte
    Anzahl_registrierte_Flugobjekte = aircraft_raw_rdd.count()

    # RDD erstellen, um die Airlines nach Anzahl der registrierten Flugzeuge zu zählen
    aircraftRDD = (
        aircraft_raw_rdd.filter(lambda line: len(line.split(",")) > 16)
        .map(lambda line: line.split(",")[17].strip('"'))
        .filter(lambda airline: airline != "''")
        .map(lambda airline: (airline, 1))
        .reduceByKey(lambda a, b: a + b)
        .sortBy(lambda x: x[1], ascending=False)
    )
    
    # Top 20 Airlines nach Anzahl der registrierten Flugzeuge
    top_20_airlines = aircraftRDD.take(20)

    # Ergebnis als formatierten String zurückgeben
    result = f"Anzahl der weltweit gemeldeten, eindeutigen Flug- und Bodenobjekte: {Anzahl_registrierte_Flugobjekte}\n"
    result += "=" * 50 + "\n"
    result += "Top 20 Airlines nach Anzahl der registrierten Flugzeuge:\n"
    result += "=" * 50 + "\n"

    for airline, count in top_20_airlines:
        result += f"{airline}: {count}\n"

    return result, aircraft_raw_rdd, aircraftRDD

# Beispielaufruf und Ausgabe
result, aircraft_raw_rdd, aircraftRDD = analyze_aircraft_data()
print(result)


Anzahl der weltweit gemeldeten, eindeutigen Flug- und Bodenobjekte: 601337
Top 20 Airlines nach Anzahl der registrierten Flugzeuge:
United States Air Force: 2223
Delta Air Lines: 781
American Airlines: 657
Corporate Airlink: 649
Federal Express: 560
Royal Air Force: 503
United Airlines: 451
Southwest Airlines: 280
German Air Force: 277
Boeing: 274
United Parcel Service: 270
Indian Air Force: 236
Force Aerienne Francaise: 223
Gulfstream Aerospace: 201
Bombardier: 190
Ryanair: 159
Qatar Airways: 157
Skywest Airlines: 153
Aeroflot Russian Airlines: 147
Royal Netherlands Air Force: 144



Auswertung 3 - Weltweite Bewegungsdaten von Flugobjekten in einer Woche

In [4]:
import folium

# Analyse der Bewegungsdaten
def analyze_flight_movements(percentage=1.0):
    """
    Analysiert Bewegungsdaten von Flugzeugen und erstellt eine Karte des geografischen Bereichs.
    Misst dabei die Laufzeit für die Verarbeitung des Prozentsatzes der Daten.
    
    :param percentage: Prozentsatz (zwischen 0 und 1) der Daten, die geladen werden sollen (Standard: 100%)
    :return: Die Ergebnisse der Auswertung, generierte Karte und das RDD der Daten sowie die benötigte Zeit
    """

    if not 0 < percentage <= 1:
        raise ValueError("Percentage muss zwischen 0 und 1 liegen")
    
    # Einlesen der Bewegungsdaten eines definierten Zeitraums in einem bestimmten geografischen Bereich
    myFileRDD = sc.textFile('data/processed_data.csv')
    header = myFileRDD.take(2)
    sampleRDD = myFileRDD.filter(lambda line: line not in header)

    #Reduktion des RDDs auf Prozentwert an Datensätzen, seed sorgt für reproduzierbare Zufallsergebnisse (42 ist die Antwort auf alles).
    dataRDD = sampleRDD.sample(False, percentage, seed=42)

    #Erfassen des Zeitraums in dem Daten die erhoben wurden - Skalierbarkeit gegeben aufgrund der Rückgabe eines Wertes
    start_time = dataRDD.map(lambda line: line.split(',')[0]).min()
    end_time = dataRDD.map(lambda line: line.split(',')[0]).max()

    #Geographische Analyse des betrachteten Luftraums - Skalierbarkeit gegeben aufgrund der Rückgabe und Visualisierung einzelner Werte

    # Längengrade sortieren und auslesen
    min_longitude = dataRDD.map(lambda line: float(line.split(',')[3])).min()
    max_longitude = dataRDD.map(lambda line: float(line.split(',')[3])).max()

    # Breitengrade (vermutlich Spalte mit Index 5 und 6)
    min_latitude = dataRDD.map(lambda line: float(line.split(',')[2])).min()
    max_latitude = dataRDD.map(lambda line: float(line.split(',')[2])).max()

    # Zentrum des Bereichs berechnen
    center_lat = (min_latitude + max_latitude) / 2
    center_lon = (min_longitude + max_longitude) / 2
    result_text = (
        f"Die erfassten Bewegungen von Flug- und Bodenobjekten wurden im Zeitraum vom "
        f"{start_time.replace('\"', '').replace('+00', '')} bis zum "
        f"{end_time.replace('\"', '').replace('+00', '')} (Zeitzone: UTC) ermittelt.\n"
        f"{'=' * 50}\n"
        "Geografische Abdeckung:\n"
        f"Längengrade: von {min_longitude:.4f}° bis {max_longitude:.4f}°\n"
        f"Breitengrade: von {min_latitude:.4f}° bis {max_latitude:.4f}°\n"
        "Die Daten wurden in folgendem Luftraum erhoben."
    )

    # Karte erstellen
    map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Rechteck auf der Karte hinzufügen
    bounds = [[min_latitude, min_longitude], [max_latitude, max_longitude]]
    folium.Rectangle(
        bounds=bounds,
        color="red",
        weight=2,
        fill=True,
        fill_color="blue",
        fill_opacity=0.2,
        popup="Geographischer Bereich"
    ).add_to(map)

    return result_text, map, dataRDD,  

result_text, map_result, dataRDD = analyze_flight_movements()
print(result_text)
map_result

25/01/03 11:57:11 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Die erfassten Bewegungen von Flug- und Bodenobjekten wurden im Zeitraum vom 2024/11/19 10:36:44 bis zum 2024/11/26 13:59:59 (Zeitzone: UTC) ermittelt.
Geografische Abdeckung:
Längengrade: von -175.5147° bis 177.9875°
Breitengrade: von -46.4715° bis 69.9645°
Die Daten wurden in folgendem Luftraum erhoben.


Auswertung 4 - Kombination Flugobjekte und Bewegungsdaten

In [5]:
def restructure_flight_movements(dataRDD):
    """
    Analyse der Bewegungsdaten nach Flug- und Bodenobjekten.

    Args:
        dataRDD: Ein RDD mit Roh-Bewegungsdaten.
        percentage: Prozentsatz der Daten, die analysiert werden sollen.

    Returns:
        Ein transformierter RDD, gruppiert nach Flugzeug-ID, und die Anzahl der aktiven Flugzeuge.
    """
    def parse_line(line):
        parts = line.split(',')
        return (
            parts[1].strip(' "\t\r\n'),  # Key: Flugzeug-ID (icao24)
            {  # Value: Alle relevanten Daten als Dictionary
                "time": parts[0].strip('"'),
                "lat": float(parts[2].strip('"')),
                "lon": float(parts[3].strip('"')),
                "onground": int(parts[4].strip('"'))
            }
        )
    
    # RDD transformieren: Gruppiere die Daten nach Flugzeug-ID
    flights_rdd = dataRDD.map(parse_line).groupByKey().mapValues(list)

    return flights_rdd

flights_rdd = restructure_flight_movements(dataRDD)

In [6]:
# Kombination der Bewegungsdaten mit den Daten der registrierten Flug- und Bodenobjekten

def combine_rdds(flights_rdd, aircraft_raw_rdd):
    """
    Kombiniert die Bewegungsdaten (flights_rdd) mit den Daten der registrierten Flug- und Bodenobjekte (aircraft_raw_rdd).

    Args:
        flights_rdd: Ein RDD mit Bewegungsdaten.
        aircraft_raw_rdd: Ein RDD mit Rohdaten zu registrierten Flug- und Bodenobjekten.

    Returns:
        Ein kombiniertes RDD mit Flugzeug-ID, Flugdaten und Beschreibungen.
    """
    def parse_type(line):
        parts = line.split(',')
        # Baseline-Verarbeitung ohne unnötige Komplexität
        return (
            parts[0].strip("'"),  # Flugzeug-ID (icao24)
            {
                "description": parts[5].strip("'") if len(parts) > 4 else "",
                "manufacturer": parts[13].strip("'") if len(parts) > 12 else "",
                "model": parts[14].strip("'") if len(parts) > 14 else "",
                "type": parts[15].strip("'") if len(parts) > 15 else "",
                "airline": parts[18].strip("'") if len(parts) > 16 else ""
            }
        )

    # RDD bereinigen und nur relevante Daten aufnehmen
    cleaned_type_rdd = aircraft_raw_rdd.map(parse_type)

    # Kombination der Bewegungsdaten mit der Beschreibung der registrierten Flug- und Bodenobjekten
    result_rdd = flights_rdd.leftOuterJoin(cleaned_type_rdd)

    # Kombination der Daten in einem RDD
    combined_rdd = result_rdd.map(lambda x: (
        x[0],  # Flugzeug-ID
        {
            "aircraft_info": x[1][1] if x[1][1] else {},  # Beschreibung, falls vorhanden
            "flight_data": sorted(x[1][0], key=lambda entry: entry["time"]) if x[1][0] else [] # Flugdaten sortiert nach Datum (lexografisch)
        }
    ))

    return combined_rdd


combined_rdd = combine_rdds(flights_rdd, aircraft_raw_rdd)
#combined_rdd.take(1)

In [ ]:
#Analyse des kombinierten RDDs (Bewegungsdaten + Beschreibung)

def analyze_combined_rdd(sampleRDD, percentage=1.0):
    """
    Analysiert ein kombiniertes RDD, das Bewegungsdaten von Flug- und Bodenobjekten enthält,
    und liefert eine detaillierte statistische Zusammenfassung der betrachteten Daten.

    Args:
    sampleRDD (RDD): Ein RDD, das Flug- und Bewegungsdaten von Objekten enthält, wobei jedes Element ein Tupel ist,
                      wobei das erste Element die Objekt-ID ist und das zweite Element ein Dictionary mit den Bewegungsdaten.
    percentage (float): Der Anteil der Stichprobe (zwischen 0 und 1) der RDD-Daten, die für die Analyse verwendet werden sollen. 
                        Standardwert ist 1.0 (100%).

    Returns:
    str: Eine formatierte Zeichenkette, die die Ergebnisse der Analyse zusammenfasst, einschließlich der Gesamtzahl der
         Objekte, der Flugzeuge mit den meisten Datensätzen und der Status- und Bewegungsanalyse für den betrachteten Zeitraum.

    """
    
    #Reduktion des RDDs auf Prozentwert an Datensätzen, seed sorgt für reproduzierbare Zufallsergebnisse (42 ist die Antwort auf alles).
    combined_rdd = sampleRDD.sample(False, percentage, seed=42)
    amount_of_aircrafts = combined_rdd.count()
    result = "=" * 50 + "\n"
    result += f"Anzahl der im betrachteten Zeitraum aktiven Flug- und Bodenobjekte: {amount_of_aircrafts}\n"
    
    #Ermittlung der Anzahl an Flug-Datensätze pro Flugzeug
    countdatasetsRDD = combined_rdd.map(lambda x: (x[0],len(x[1]['flight_data']))).sortBy(lambda x: x[1], ascending=False)
    first_5_elements = countdatasetsRDD.take(5)
    result += "=" * 50 + "\n"
    result += f"Flugzeuge mit den meisten Datensätzen (Top 5) sind: {first_5_elements}\n "
    result += "=" * 50 + "\n"


    # Bestimmung der Positon - Skalierbarkeit gegeben über Betrachtung einzelner Werte
    airRDD = combined_rdd.map(lambda x: (x[0], x[1]['flight_data'][-1]["onground"]))
    onground = airRDD.filter(lambda x: x[1]==1).count()
    inair = airRDD.filter(lambda x: x[1]==0).count()
    result += f"Status der Objekt zum letzten gemessenen Zeitpunkt: \n"
    result += f"Objekte aktuell in der Luft: {onground}\n"
    result += f"Objekte aktuell am Boden: {inair}\n"


    # Ermittlung der Position im Zeitraum - Skalierbarkeit gegeben über Betrachtung einzelner Werte
    nextRDD = combined_rdd.map(lambda x: (x[0], sum(datapoint["onground"] for datapoint in x[1]['flight_data']), len(x[1]['flight_data'])))
    calculatedRDD = nextRDD.map(lambda x: (x[0], x[1]/x[2]))
    startet_or_landed = calculatedRDD.filter(lambda x: x[1] < 1 and x[1] >0).count()
    not_started = calculatedRDD.filter(lambda x: x[1] == 1).count()
    not_landed = calculatedRDD.filter(lambda x: x[1] == 0).count()

    result += "=" * 50 + "\n"
    result += f"Im gesamten betrachteten Zeitraum sind: \n"
    result += f"Objekte nur am Boden geblieben: {not_started}\n"
    result += f"Objekte nur in der Luft geblieben: {not_landed}\n"
    result += f"Objekte gestartet oder gelandet: {startet_or_landed}"


    return combined_rdd, result

combined_analyzed_rdd, result = analyze_combined_rdd(combined_rdd)
print(result)

Anzahl der im betrachteten Zeitraum aktiven Flug- und Bodenobjekte: 12104
Flugzeuge mit den meisten Datensätzen (Top 5) sind: [('474805', 21485), ('440cab', 19321), ('440cac', 17565), ('ad976e', 17409), ('474809', 17093)]
Status der Objekt zum letzten gemessenen Zeitpunkt: 
Objekte aktuell in der Luft: 2050
Objekte aktuell am Boden: 10054
Im gesamten betrachteten Zeitraum sind: 
Objekte nur am Boden geblieben: 675
Objekte nur in der Luft geblieben: 7780
Objekte gestartet oder gelandet: 3649


==== Visuelle Analysen ====

In [ ]:
import math

# Haversine-Funktion zum Berechnen der Distanz zwischen zwei Punkten
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Erd-Radius in Kilometern
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat / 2.0) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def find_nearest_airport(lat, lon, airports):
    """
    Findet den nächsten Flughafen zu gegebenen Koordinaten innerhalb von 10 km.
    Gibt den Flughafen mit der geringsten Distanz zurück.
    """
    nearest_airport = None
    min_distance = float("inf")

    for airport in airports:
        airport_id, _, airport_name, airport_lat, airport_lon = airport
        airport_lat = float(airport_lat)
        airport_lon = float(airport_lon)
        distance = haversine(lat, lon, airport_lat, airport_lon)
        if distance <= 10 and distance < min_distance:  # Innerhalb von 10 km und geringste Distanz
            min_distance = distance
            nearest_airport = (airport_id, airport_name)

    if nearest_airport:
        return nearest_airport
    return None, None

def extract_flights_with_airports(record, airports):
    """
    Extrahiert Flüge und fügt Flughafendaten hinzu.
    """
    icao24, combined_rdd = record
    aircraft_info = combined_rdd['aircraft_info']
    flight_data = combined_rdd['flight_data']

    # Filter: Ignoriere Datensätze mit "Surface Vehicle" in der Beschreibung
    if 'description' in aircraft_info and 'Surface Vehicle' in aircraft_info['description']:
        return []

    flights = []
    current_flight = []
    in_flight = False
    previous_entry = None  # Variable, um den vorherigen Datensatz zu speichern

    for entry in flight_data:
        # Überprüfung, ob es sich um einen Start handelt
        if not in_flight and entry['onground'] == 0 and previous_entry and previous_entry['onground'] == 1:
            in_flight = True
            current_flight = [entry]  # Startpunkt hinzufügen
        elif in_flight and entry['onground'] == 1:  # Landung erkannt
            current_flight.append(entry)  # Landepunkt hinzufügen
            # Flug-Informationen extrahieren
            start = current_flight[0]
            end = current_flight[-1]

            start_airport_id, start_airport_name = find_nearest_airport(start['lat'], start['lon'], airports)
            end_airport_id, end_airport_name = find_nearest_airport(end['lat'], end['lon'], airports)

            flight_info = {
                'start_time': start['time'],
                'start_coords': (start['lat'], start['lon']),
                'start_airport': {'id': start_airport_id, 'name': start_airport_name},
                'end_time': end['time'],
                'end_coords': (end['lat'], end['lon']),
                'end_airport': {'id': end_airport_id, 'name': end_airport_name},
            }
            flights.append((icao24, aircraft_info, flight_info))
            in_flight = False
            current_flight = []
        elif in_flight:  # Punkte während des Fluges sammeln
            current_flight.append(entry)

        # Speichere den aktuellen Eintrag als vorherigen für die nächste Iteration
        previous_entry = entry

    return flights

# Airports-Daten als Liste sammeln (Broadcast-Variable verwenden)
# Annahme: airportRDD ist das gegebene RDD mit den Flughafendaten
airports_list = airportRDD.collect()
broadcast_airports = sc.broadcast(airports_list)

# Anwenden der Extraktionsfunktion auf die RDD mit Flughafendaten
flights_rdd = combined_rdd.flatMap(lambda record: extract_flights_with_airports(record, broadcast_airports.value))

# Sammle alle Daten aus dem RDD
all_flights = flights_rdd.collect()

# Gib alle gesammelten Flugdaten aus
for flight in all_flights:
    print(flight)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.graph_objects as go

# Beispiel-Daten erweitern: Liste von Flügen
flights_data = flights_rdd.collect()

# Schritt 1: Flugverbindungen aggregieren (Start- und End-Airport-Paare zählen)
connections = defaultdict(int)
start_end_names = {}

for _, _, flight_info in flights_data:
    start_airport = flight_info['start_airport']
    end_airport = flight_info['end_airport']
    
    if start_airport['id'] and end_airport['id']:  # Nur valide Airports berücksichtigen
        start_id, start_name = start_airport['id'], start_airport['name']
        end_id, end_name = end_airport['id'], end_airport['name']
        
        connections[(start_id, end_id)] += 1
        start_end_names[start_id] = start_name
        start_end_names[end_id] = end_name

# Schritt 2: Netzwerkdiagramm erstellen
G = nx.Graph()

# Knoten und Kanten hinzufügen (nur Verbindungen mit mehr als 5 Flügen)
for (start, end), weight in connections.items():
    if weight > 5:  # Filter: Nur Verbindungen > 5
        G.add_edge(start, end, weight=weight)

# Schritt 3: Positionen berechnen
pos = nx.spring_layout(G, seed=42)

# Knoten und Kanten für Plotly vorbereiten
edges_x = []
edges_y = []
weights = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edges_x.extend([x0, x1, None])
    edges_y.extend([y0, y1, None])
    weights.append(edge[2]['weight'])

# Knoten vorbereiten
nodes_x = []
nodes_y = []
node_text = []

for node in G.nodes():
    x, y = pos[node]
    nodes_x.append(x)
    nodes_y.append(y)
    node_text.append(start_end_names.get(node, node))  # Namen verwenden

# Schritt 4: Interaktives Diagramm mit Plotly erstellen
fig = go.Figure()

# Kanten hinzufügen mit stärkerer Gewichtung
max_weight = max(weights) if weights else 1  # Maximale Gewichtung für Skalierung

for i, edge in enumerate(G.edges(data=True)):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        line=dict(width=(weights[i] / max_weight) * 100, color='black'),  # Stärkere Skalierung (100-fach)
        hoverinfo='none',
        mode='lines'))

# Knoten hinzufügen
fig.add_trace(go.Scatter(
    x=nodes_x, y=nodes_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    marker=dict(
        size=10,
        color='skyblue',
        line_width=2),
    hoverinfo='text'))

# Layout anpassen
fig.update_layout(
    title="Interaktives Netzwerkdiagramm der Flugverbindungen (Verbindungen > 5)",
    title_x=0.5,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=30),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white'
)

# Interaktives Diagramm anzeigen
fig.show()


Visualisierung der Bewegung und Beschreibung eines ausgewählten Flug- oder Bodenobjektes

In [ ]:
import folium
from IPython.display import display

def analyze_aircraft_route(selected_icao):
    # Filtere die Daten für die spezifische Flugzeug-ID im RDD
    filtered_data = combined_rdd.filter(lambda x: x[0] == selected_icao).collect()

    if filtered_data:
        # Hole die Daten für das ausgewählte Flugzeug
        icao24, route_data = filtered_data[0]
        aircraft_info = route_data["aircraft_info"]  # Zugriff auf die Flugzeugbeschreibung
        flight_data = route_data["flight_data"]  # Zugriff auf die Flugdaten

        print(f"Flugzeug-ID: {icao24}")

        # Erstelle die Karte, zentriert auf den ersten Datenpunkt
        first_lat = flight_data[0]["lat"]
        first_lon = flight_data[0]["lon"]
        map = folium.Map(location=[first_lat, first_lon], zoom_start=12)

        # Zeichne Linien zwischen aufeinanderfolgenden Punkten
        for i in range(len(flight_data) - 1):
            point_a = flight_data[i]
            point_b = flight_data[i + 1]

            # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
            if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
                folium.PolyLine(
                    locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                    color="blue",
                    weight=2.5,
                    opacity=0.8
                ).add_to(map)

        # Füge Marker nur für Landepunkte hinzu
        for point in flight_data:
            if point["onground"] == 1:  # Nur Landepunkte anzeigen
                folium.Marker(
                    location=(point["lat"], point["lon"]),
                    popup=f"Zeit: {point['time']}<br>Am Boden: {point['onground']}<br>Hersteller: {aircraft_info['manufacturer']}<br>Flugzeugtyp: {aircraft_info['model']}<br>Airline: {aircraft_info['airline']}",
                    icon=folium.Icon(color="green", icon="info-sign")
                ).add_to(map)

        # Rückgabe der Karte
        return map

    else:
        return f"Keine Daten für Flugzeug-ID '{selected_icao}' gefunden."


result = analyze_aircraft_route("c04fdd")
display(result)

Visualisierung der Bewegung und Beschreibung mehrerer, zufällig ausgewählter Flug- oder Bodenobjekte

In [ ]:
import folium
from IPython.display import display
import random

# Zufällige Farben für die Flugzeuge
def get_random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

def analyze_aircraft_routes(number_of_samples):
    # Nimm eine beliebige Anzahl zufälliger Flugzeuge aus dem RDD
    random_aircrafts = combined_rdd.takeSample(withReplacement=False, num=number_of_samples)

    # Erstelle eine Karte, zentriert auf einen Beispielpunkt
    if random_aircrafts:
        # Hole den ersten Punkt zur Zentrierung der Karte
        first_lat = random_aircrafts[0][1]["flight_data"][0]["lat"]
        first_lon = random_aircrafts[0][1]["flight_data"][0]["lon"]
        map = folium.Map(location=[first_lat, first_lon], zoom_start=6)

        # Iteriere über die zufälligen Flugzeuge
        for icao24, data in random_aircrafts:
            aircraft_info = data["aircraft_info"]  # Infos über das Flugzeug
            flight_data = data["flight_data"]      # Flugdaten

            # Sichere Abfrage von Typ, Modell und Hersteller mit Standardwerten
            aircraft_type = aircraft_info.get("type", "Unknown")
            aircraft_model = aircraft_info.get("model", "Unknown")
            aircraft_manufacturer = aircraft_info.get("manufacturer", "Unknown")
            aircraft_airline = aircraft_info.get("airline", "Unknown")

            # Generiere eine zufällige Farbe für die Linien
            color = get_random_color()

            # Zeichne Linien zwischen aufeinanderfolgenden Punkten
            for i in range(len(flight_data) - 1):
                point_a = flight_data[i]
                point_b = flight_data[i + 1]

                # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
                if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
                    folium.PolyLine(
                        locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                        color=color,
                        weight=2.5,
                        opacity=0.8
                    ).add_to(map)

            # Füge Marker nur für Landepunkte hinzu
            for point in flight_data:
                if point["onground"] == 1:  # Nur Landepunkte anzeigen
                    folium.Marker(
                        location=(point["lat"], point["lon"]),
                        popup=(
                            f"Flugzeug: {icao24}<br>"
                            f"Hersteller: {aircraft_manufacturer}<br>"
                            f"Modell: {aircraft_model}<br>"
                            f"Airline: {aircraft_airline}<br>"
                            f"Zeit: {point['time']}<br>"
                            f"Am Boden: {point['onground']}"
                        ),
                        icon=folium.Icon(color="green", icon="info-sign")
                    ).add_to(map)

        # Zeige die Karte im Notebook
        return map
    else:
        return "Keine Daten gefunden."
    
result = analyze_aircraft_routes(20)
display(result)
    

Heatmap zur visuellen Auswertung aller Bewegungsdaten

In [ ]:
import folium
from folium.plugins import HeatMap

# Map und Filter: Direktes Überprüfen der Konvertierbarkeit in float
# 1 Dezimalstelle: ~11,1 km Genauigkeit eine Nachkommastelle reicht für globale Karten oder grobe Cluster.
# 2 Dezimalstellen: ~1,1 km Genauigkeit gut für Städte oder Regionen.
# 3 Dezimalstellen: ~110 m Genauigkeit ideal für Stadtviertel oder grobe Stadtanalysen.
# Ab 3 Nachkommastellen geht nix mehr

def extract_coordinates(line):
    try:
        parts = line.split(',')
        return round(float(parts[3]), 2), round(float(parts[4]), 2)  # Breitengrad, Längengrad
    except (ValueError, IndexError):
        return None  # Falls Konvertierung oder Zugriff fehlschlägt

location_rdd = dataRDD.map(extract_coordinates).filter(lambda x: x is not None)

number = location_rdd.count()
print(f"Anzahl der Daten vor der Reduktion {number}")


# Map-Reduce: Zähle, wie oft jede Kombination von Breitengrad und Längengrad vorkommt
coordinate_counts = location_rdd.map(lambda coord: (coord, 1)).reduceByKey(lambda a, b: a + b)

# Ergebnisse anzeigen
print(coordinate_counts.take(5))
print(f"Anzahl Koordinaten nach der Reduktion {coordinate_counts.count()}")

# Beispiel: Aggregierte Koordinaten mit Häufigkeiten (ersetzt durch deine Daten)
aggregated_data = coordinate_counts.collect()

# Vorbereitung der Heatmap-Daten
heatmap_data = [(lat_lon[0], lat_lon[1], count) for lat_lon, count in aggregated_data]

# Erstelle die Karte
if heatmap_data:
    # Zentriere die Karte auf den ersten Punkt und passe die Zoom-Stufe an
    map = folium.Map(location=[heatmap_data[0][0], heatmap_data[0][1]], zoom_start=6)

    # HeatMap hinzufügen (Radius und max_zoom können angepasst werden)
    HeatMap(heatmap_data, radius=15, max_zoom=13).add_to(map)

    # Karte anzeigen
    display(map)
else:
    print("Keine gültigen Punkte für die Heatmap gefunden.")

==== Untersuchung der Analysen ====

Definition der Analyse- und Fehlertoleranz-Funktionen

In [20]:
import time
import psutil
import datetime
from multiprocessing import Process
import os

def get_system_info():
    # CPU-Informationen
    cpu_count = psutil.cpu_count(logical=False)  # Anzahl physikalischer CPU-Kerne
    logical_cpu_count = psutil.cpu_count(logical=True)  # Anzahl logischer CPUs (mit Hyper-Threading)
    cpu_freq = psutil.cpu_freq()  # CPU Frequenz
    cpu_percent = psutil.cpu_percent(interval=1)  # CPU-Auslastung in Prozent

    # RAM-Informationen
    virtual_memory = psutil.virtual_memory()  # Virtueller Speicher (RAM)
    total_memory = virtual_memory.total  # Gesamtgröße des RAM
    available_memory = virtual_memory.available  # Verfügbarer RAM
    used_memory = virtual_memory.used  # Genutzter RAM
    memory_percent = virtual_memory.percent  # Prozentsatz des verwendeten RAM


    # Ausgabe der gesammelten Informationen
    result = "=" * 50
    result += "System Informationsübersicht"
    result += "=" * 50\

    result += "\nCPU Infos:\n"
    result += f"  - Anzahl der physischen CPU-Kerne: {cpu_count}\n"
    result += f"  - Anzahl der logischen CPU-Kerne (inkl. Hyper-Threading): {logical_cpu_count}\n"
    result += f"  - Aktuelle CPU Frequenz: {cpu_freq.current} MHz\n"
    result += f"  - Aktuelle CPU Auslastung: {cpu_percent}%\n"

    result += "\nRAM Infos:\n"
    result += f"  - Gesamter RAM: {total_memory / (1024 ** 3):.2f} GB\n"
    result += f"  - Verfügbarer RAM: {available_memory / (1024 ** 3):.2f} GB\n"
    result += f"  - Genutzter RAM: {used_memory / (1024 ** 3):.2f} GB\n"
    result += f"  - RAM Auslastung: {memory_percent}%"

    return result



# Datenverteilung im CLuster
def analyze_partition_distribution(rdd):
    """
    Analysiert die Verteilung der Daten in einem RDD über die Partitionen.

    :param rdd: Das RDD, das analysiert werden soll.
    """
    # Funktion, um die Größe jeder Partition zu berechnen
    def partition_sizes(index, iterator):
        yield index, sum(1 for _ in iterator)

   # Daten je Partition sammeln
    partition_info = rdd.mapPartitionsWithIndex(partition_sizes).collect()

    # Anzahl der Partitionen ermitteln
    num_partitions = rdd.getNumPartitions()

    #Ergebnisse formatieren
    number_data = f"Anzahl der Datensätze: {rdd.count()}\n"
    number_partition = f"Anzahl der Partitionen: {num_partitions}\n"
    result = "Datenverteilung auf Partitionen:\n"

    # Ausgabe der Verteilung pro Partition
    for partition, size in partition_info:
        result += f"Partition {partition}: {size} Datensätze\n"

    return number_data, number_partition, result


# Fehlertoleranz

# Prozess, der die CPU lokal belastet
def cpu_stress():
    while True:
        pass

if __name__ == "__main__":
    cpu_stress()


# Simulieren von Netzwerkfehlern (Ausfall der Kommunikation im Netzwerk)
def simulate_network_failure():
    # Blockiere Loopback-Kommunikation (127.0.0.1), die Spark nutzt
    os.system("iptables -A INPUT -s 127.0.0.1 -j DROP")

def restore_network():
    # Entferne die Blockade
    os.system("iptables -D INPUT -s 127.0.0.1 -j DROP")



KeyboardInterrupt: 

Hardwareanalyse

In [10]:
# Hardwareanalyse
print(get_system_info())

==================================================System Informationsübersicht==================================================
CPU Infos:
  - Anzahl der physischen CPU-Kerne: 12
  - Anzahl der logischen CPU-Kerne (inkl. Hyper-Threading): 12
  - Aktuelle CPU Frequenz: 4056 MHz
  - Aktuelle CPU Auslastung: 11.3%

RAM Infos:
  - Gesamter RAM: 36.00 GB
  - Verfügbarer RAM: 10.06 GB
  - Genutzter RAM: 17.58 GB
  - RAM Auslastung: 72.1%


Analyse 1 - Europäische Flughäfen

In [11]:
import time

#Laufzeit
start_time = time.time()
analyze_airports()
end_time = time.time()
print(f"Die Funktion hat {(end_time - start_time):.4f} Sekunden benötigt")

#Datenverteilung
number_data, number_partition, result = analyze_partition_distribution(airportRDD)
print(number_data + number_partition + "\n" + result)

Die Funktion hat 0.3296 Sekunden benötigt
Anzahl der Datensätze: 67961
Anzahl der Partitionen: 2

Datenverteilung auf Partitionen:
Partition 0: 34380 Datensätze
Partition 1: 33581 Datensätze



Analyse 2 - Flugzeugbeschreibungen

In [12]:
import time

#Laufzeit
start_time = time.time()
analyze_aircraft_data()
end_time = time.time()
print(f"Die Funktion hat {(end_time - start_time):.4f} Sekunden benötigt")

#Datenverteilung
number_data, number_partition, result = analyze_partition_distribution(aircraftRDD)
print(number_data + number_partition + "\n" + result)

Die Funktion hat 0.8380 Sekunden benötigt
Anzahl der Datensätze: 4581
Anzahl der Partitionen: 4

Datenverteilung auf Partitionen:
Partition 0: 1084 Datensätze
Partition 1: 880 Datensätze
Partition 2: 0 Datensätze
Partition 3: 2617 Datensätze



Analyse 3 - Bewegungsdaten

In [ ]:
import time

#Laufzeit
start_time = time.time()
result_text, map_result, dataRDD = analyze_flight_movements(percentage=1.0)
end_time = time.time()
print(f"Die Funktion hat für 100% der Daten {(end_time - start_time):.4f} Sekunden benötigt")
print(result_text)
print()

#Datenverteilung
number_data, number_partition, result = analyze_partition_distribution(dataRDD)
print(number_data + number_partition + result)


#Skalierbarkeit
percentages = [0.1, 0.2, 0.5, 1.0]  # 10%, 20%, 50%, 100%

for percentage in percentages:
    print(f"\nVerarbeite {percentage*100}% der Daten...")
    start_time = time.time()
    result_text, map_result, dataRDD = analyze_flight_movements(percentage=percentage)
    end_time = time.time()
    
    print(f"Verarbeitungszeit für {percentage*100}% der Daten: {(end_time-start_time):.2f} Sekunden")
    number_data, number_partition, result = analyze_partition_distribution(dataRDD)
    print(number_data + number_partition)
    print(f"Anzahl der Worker Nodes: {len(sc._jsc.sc().statusTracker().getExecutorInfos()) - 1}")  # -1 für den Treiber
    print(f"Gesamte Anzahl der Kerne: {sc.defaultParallelism}")

# Fehlertoleranz



Analyse 4 - Kombinierte Flugobekte und Bewegungsdaten

In [25]:
import time

#Laufzeit
start_time = time.time()
result_text, map_result, dataRDD = analyze_flight_movements()
flights_rdd = restructure_flight_movements(dataRDD)    
result, aircraft_raw_rdd, aircraftRDD = analyze_aircraft_data()
combined_rdd = combine_rdds(flights_rdd, aircraft_raw_rdd)
combined_analyzed_rdd, result = analyze_combined_rdd(combined_rdd)
end_time = time.time()

print(f"Die Funktion hat für 100% der Daten {(end_time - start_time):.4f} Sekunden benötigt")
print(result)
print()

#Datenverteilung
number_data, number_partition, result = analyze_partition_distribution(combined_analyzed_rdd)
print(number_data + number_partition + result)


#Skalierbarkeit
percentages = [0.1, 0.2, 0.5, 1.0]  # 10%, 20%, 50%, 100%

for percentage in percentages:
    print(f"\nVerarbeite {percentage*100}% der Daten...")
    start_time = time.time()
    result_text, map_result, dataRDD = analyze_flight_movements(percentage=percentage)
    flights_rdd = restructure_flight_movements(dataRDD)
    result, aircraft_raw_rdd, aircraftRDD = analyze_aircraft_data()
    combined_rdd = combine_rdds(flights_rdd, aircraft_raw_rdd)
    combined_analyzed_rdd, result = analyze_combined_rdd(combined_rdd, percentage)
    end_time = time.time()
    
    print(f"Verarbeitungszeit für {percentage*100}% der Daten: {(end_time-start_time):.2f} Sekunden")
    number_data, number_partition, result = analyze_partition_distribution(combined_analyzed_rdd)
    print(number_data + number_partition)
    print(f"Anzahl der Worker Nodes: {len(sc._jsc.sc().statusTracker().getExecutorInfos()) - 1}")  # -1 für den Treiber
    print(f"Gesamte Anzahl der Kerne: {sc.defaultParallelism}")

#Fehlertoleranz


Die Funktion hat für 100% der Daten 57.2920 Sekunden benötigt
Anzahl der im betrachteten Zeitraum aktiven Flug- und Bodenobjekte: 12104
Flugzeuge mit den meisten Datensätzen (Top 5) sind: [('474805', 21485), ('440cab', 19321), ('440cac', 17565), ('ad976e', 17409), ('474809', 17093)]
Status der Objekt zum letzten gemessenen Zeitpunkt: 
Objekte aktuell in der Luft: 2050
Objekte aktuell am Boden: 10054
Im gesamten betrachteten Zeitraum sind: 
Objekte nur am Boden geblieben: 675
Objekte nur in der Luft geblieben: 7780
Objekte gestartet oder gelandet: 3649



Anzahl der Datensätze: 12104
Anzahl der Partitionen: 65
Datenverteilung auf Partitionen:
Partition 0: 206 Datensätze
Partition 1: 184 Datensätze
Partition 2: 229 Datensätze
Partition 3: 195 Datensätze
Partition 4: 182 Datensätze
Partition 5: 189 Datensätze
Partition 6: 210 Datensätze
Partition 7: 173 Datensätze
Partition 8: 198 Datensätze
Partition 9: 187 Datensätze
Partition 10: 186 Datensätze
Partition 11: 203 Datensätze
Partition 12: 178 Datensätze
Partition 13: 177 Datensätze
Partition 14: 211 Datensätze
Partition 15: 163 Datensätze
Partition 16: 191 Datensätze
Partition 17: 184 Datensätze
Partition 18: 185 Datensätze
Partition 19: 197 Datensätze
Partition 20: 198 Datensätze
Partition 21: 185 Datensätze
Partition 22: 178 Datensätze
Partition 23: 192 Datensätze
Partition 24: 155 Datensätze
Partition 25: 181 Datensätze
Partition 26: 174 Datensätze
Partition 27: 163 Datensätze
Partition 28: 199 Datensätze
Partition 29: 171 Datensätze
Partition 30: 178 Datensätze
Partition 31: 177 Date

Verarbeitungszeit für 10.0% der Daten: 28.63 Sekunden
Anzahl der Datensätze: 1266
Anzahl der Partitionen: 65

Anzahl der Worker Nodes: 0
Gesamte Anzahl der Kerne: 12

Verarbeite 20.0% der Daten...


Verarbeitungszeit für 20.0% der Daten: 33.01 Sekunden


Anzahl der Datensätze: 2475
Anzahl der Partitionen: 65

Anzahl der Worker Nodes: 0
Gesamte Anzahl der Kerne: 12

Verarbeite 50.0% der Daten...


Verarbeitungszeit für 50.0% der Daten: 43.83 Sekunden


Anzahl der Datensätze: 6106
Anzahl der Partitionen: 65

Anzahl der Worker Nodes: 0
Gesamte Anzahl der Kerne: 12

Verarbeite 100.0% der Daten...


Verarbeitungszeit für 100.0% der Daten: 59.21 Sekunden


Anzahl der Datensätze: 12104
Anzahl der Partitionen: 65

Anzahl der Worker Nodes: 0
Gesamte Anzahl der Kerne: 12


==== Test Parquet ====

In [58]:
#from pyspark.sql import SparkSession

#spark = SparkSession.builder \
#    .appName("Inspect Parquet") \
#    .getOrCreate()

#df = spark.read.parquet("data/bigDataSet_v2.parquet")
#df.show(2)  # Zeigt die ersten 5 Zeilen der Parquet-Datei

#df.printSchema()
#print("Spaltennamen:", df.columns)

# Umwandeln des DataFrames in ein RDD
#rdd = df.rdd

# Bereinigen der Row-Daten und Zugreifen auf die richtigen Felder
#rdd_values = rdd.map(lambda row: (row[0].strip(), row[1], row[2], row[3]))

# Ausgabe der ersten 3 Zeilen
#print(rdd_values.take(3))